In [1]:
import os
import time
import random
import numpy as np
import matplotlib.pyplot as plt
import pybullet_envs_gymnasium
import gymnasium as gym
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from gymnasium import wrappers
from collections import deque

Initialize Experience Replay

In [2]:
class ReplayBuffer(object):
    def __init__(self, max_size = 1e6):
        self.storage = []
        self.max_size = max_size
        self.ptr = 0 # initial index

    # The add function adds new transitions to the memory: In case of overflow, new transition will be added in the beginning
    def add(self, transition):
        if len(self.storage == self.max_size):
            self.storage[int(self.ptr)] = transition
            self.ptr = (self.ptr + 1) % self.max_size
        else:
            self.storage.append(transition)

    def sample(self, batch_size):
        ind = np.random.randint(0, len(self.storage), size=batch_size)
        # initialize different batches
        batch_states, batch_next_states, batch_actions, batch_rewards, batch_dones = [], [], [], [], []
        for i in ind:
            state, next_state, action, reward, done = self.storage[i]
            batch_states.append(np.array(state, copy=False))
            batch_next_states.append(np.array(next_state, copy=False))
            batch_actions.append(np.array(action, copy=False))
            batch_rewards.append(np.array(reward, copy=False))
            batch_dones.append(np.array(done, copy=False))
        return np.array(batch_states), np.array(batch_next_states), np.array(batch_actions), np.array(batch_rewards).reshape(-1, 1), np.array(batch_dones).reshape(-1, 1)


Create one NN for Actor Model and one NN for Actor Target

In [3]:
class Actor(nn.Module):
    def __init__(self, state_dim, action_dim, max_action): # the max action variable is used to clip the actions to a range
        super(Actor, self).__init__()
        self.layer_1 = nn.Linear(state_dim, 400)
        self.layer_2 = nn.Linear(400, 300)
        self.layer_3 = nn.Linear(300, action_dim)
        self.max_action = max_action

    def forward(self, x): # function that forward propagates the signal
        x = F.relu(self.layer_1(x))
        x = F.relu(self.layer_2(x))
        x = torch.tanh(self.layer_3(x)) * self.max_action # this is done because value given by TanH is between -1 and 1.
        return x